1) Importing all the necassary packages

In [1]:
import tensorflow as tf
import zipfile
import os
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import shutil
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import Model
from sklearn.model_selection import train_test_split
import glob
import random

2)Download inception model

In [2]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

--2023-09-15 06:25:40--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.162.207, 173.194.215.207, 173.194.216.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.162.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M  85.9MB/s    in 1.0s    

2023-09-15 06:25:41 (85.9 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]



3) use training and test data from google drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')
train_dir="/content/tmp/train"
validation_dir="/content/tmp/test"
train="/content/drive/MyDrive/Colab Notebooks/train"
test="/content/drive/MyDrive/Colab Notebooks/test"

Mounted at /content/drive


3) unzip all files in to train directory because we will split them in a better proportion in the next cell which will help us have more training data

In [4]:
traincontents=os.listdir(train)
testcontents=os.listdir(test)
for item in traincontents:
  c=os.path.join(train,item)
  zip_ref = zipfile.ZipFile(c, 'r')
  zip_ref.extractall("tmp/train")
  zip_ref.close()
for item in testcontents:
  c=os.path.join(test,item)
  zip_ref = zipfile.ZipFile(c, 'r')
  zip_ref.extractall("tmp/test")
  zip_ref.close()

In [5]:
train_folder="/content/tmp/train"
test_folder = "/content/tmp/test"

4)after i merged strategy 2 and test validation data we will split it to train and test folders

In [ ]:
source_folder = "/content/tmp/train"
train_folder="/content/tmp/train"
test_folder = "/content/tmp/test"
file_list = glob.glob(os.path.join(source_folder, '**/*'), recursive=True)
random.shuffle(file_list)
train_files, test_files = train_test_split(file_list, train_size=0.8, test_size=0.2)
for file_path in train_files:
    if os.path.exists(file_path):
        destination_path = file_path.replace(source_folder, train_folder)
        os.makedirs(os.path.dirname(destination_path), exist_ok=True)
        if not os.path.exists(destination_path):
            shutil.move(file_path, destination_path)
        else:
            print(f"Destination path already exists: {destination_path}")
    else:
        print(f"File not found: {file_path}")

# Move test files to the test folder
for file_path in test_files:
    if os.path.exists(file_path):
        destination_path = file_path.replace(source_folder, test_folder)
        os.makedirs(os.path.dirname(destination_path), exist_ok=True)
        if not os.path.exists(destination_path):
            shutil.move(file_path, destination_path)
        else:
            print(f"Destination path already exists: {destination_path}")
    else:
        print(f"File not found: {file_path}")

4) used tensor flows image generator to augument the data and generate train and test data by assigning them in to batches

In [6]:
training_datagen = ImageDataGenerator(
      rescale = 1./255,
	    rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
train_generator = training_datagen.flow_from_directory(
	train_folder,
	target_size=(150,150),
	class_mode='categorical',
  batch_size=32
)
validation_datagen = ImageDataGenerator(rescale = 1./255)
validation_generator = validation_datagen.flow_from_directory(
	test_folder,
	target_size=(150,150),
	class_mode='categorical',
  batch_size=32
)

Found 2789 images belonging to 5 classes.
Found 2300 images belonging to 5 classes.


5) Loading the weights from the pretrained inception model and making them untrainable

In [7]:
local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
pre_trained_model = InceptionV3(input_shape = (150, 150, 3),
                                include_top = False,
                                weights = None)
pre_trained_model.load_weights(local_weights_file)
for layer in pre_trained_model.layers:
  layer.trainable = False

6) I took only the first few layers of the pretrained model to avoid taking layers which specialized on the specific training data

In [8]:
last_layer = pre_trained_model.get_layer('mixed5')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

last layer output shape:  (None, 7, 7, 768)


7) Adding our own layers on the pre trained model

In [9]:
# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.7)(x)
# Add a final softmax layer for classification
x = layers.Dense  (5, activation='softmax')(x)

# Append the dense network to the base model
model = Model(pre_trained_model.input, x)

# Print the model summary. See your dense network connected at the end.
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 150, 150, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 74, 74, 32)           864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 74, 74, 32)           96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 74, 74, 32)           0         ['batch_normalization[0][0

8) compile our model

In [10]:
model.compile(loss = 'categorical_crossentropy', optimizer = tf.keras.optimizers.Adam(), metrics=['accuracy'])

9) Training our model

In [12]:
# Train the model
history = model.fit(train_generator, epochs=100, steps_per_epoch=10, validation_data = validation_generator, verbose = 2, validation_steps=5)

Epoch 1/100
10/10 - 82s - loss: 0.4946 - accuracy: 0.8219 - val_loss: 3.8962 - val_accuracy: 0.4000 - 82s/epoch - 8s/step
Epoch 2/100
10/10 - 81s - loss: 0.6013 - accuracy: 0.7969 - val_loss: 4.5987 - val_accuracy: 0.2875 - 81s/epoch - 8s/step
Epoch 3/100
10/10 - 92s - loss: 0.5242 - accuracy: 0.8281 - val_loss: 3.7489 - val_accuracy: 0.4563 - 92s/epoch - 9s/step
Epoch 4/100
10/10 - 79s - loss: 0.4876 - accuracy: 0.8094 - val_loss: 4.6275 - val_accuracy: 0.4625 - 79s/epoch - 8s/step
Epoch 5/100
10/10 - 92s - loss: 0.5981 - accuracy: 0.7719 - val_loss: 4.6532 - val_accuracy: 0.4437 - 92s/epoch - 9s/step
Epoch 6/100
10/10 - 91s - loss: 0.5021 - accuracy: 0.8062 - val_loss: 4.7671 - val_accuracy: 0.4625 - 91s/epoch - 9s/step
Epoch 7/100
10/10 - 95s - loss: 0.6331 - accuracy: 0.7594 - val_loss: 4.6293 - val_accuracy: 0.2688 - 95s/epoch - 10s/step
Epoch 8/100
10/10 - 91s - loss: 0.5002 - accuracy: 0.8219 - val_loss: 4.5563 - val_accuracy: 0.3375 - 91s/epoch - 9s/step
Epoch 9/100
10/10 - 81s

KeyboardInterrupt: ignored